# Tigrigna Keyboard and Spell Checker Setup

This notebook will help you set up and use the Tigrigna keyboard and spell checker in your local Jupyter environment.

## 1. Check Required Packages

First, let's make sure you have the required packages installed.

In [ ]:
import sys
import importlib.util

# Check for required packages
required_packages = ['ipywidgets', 'notebook']
missing_packages = []

for package in required_packages:
    if importlib.util.find_spec(package) is None:
        missing_packages.append(package)

if missing_packages:
    print(f"Please install the following packages: {', '.join(missing_packages)}")
    print("You can install them using: pip install " + " ".join(missing_packages))
else:
    print("All required packages are installed!")
    
# Import packages
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re
import os

## 2. Check for Dictionary File

Now, let's check if the Tigrigna dictionary file exists.

In [ ]:
def check_dictionary_file(file_path='tigrigna_dictionary.txt'):
    """Check if the dictionary file exists and load it"""
    if not os.path.exists(file_path):
        print(f"Warning: Dictionary file '{file_path}' not found!")
        print("Please make sure the dictionary file is in the same directory as this notebook.")
        return False, set()
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            dictionary = {line.strip() for line in file if line.strip()}
        print(f"Dictionary loaded successfully with {len(dictionary)} words!")
        return True, dictionary
    except Exception as e:
        print(f"Error loading dictionary: {e}")
        return False, set()

# Check the dictionary file
dict_exists, dictionary = check_dictionary_file()

## 3. Create the Tigrigna Keyboard

Now, let's create the Tigrigna keyboard based on the layout from the reference image.

In [ ]:
def generate_variants(base_char):
    """
    Generate vowel variants for a Tigrigna consonant
    
    Args:
        base_char (str): The base consonant character
        
    Returns:
        list: List of vowel variants for the character
    """
    if base_char == ' ' or not base_char or len(base_char) != 1:
        return [base_char]
    
    try:
        base_code = ord(base_char)
        # In Ethiopic scripts like Tigrigna, vowel variants follow a pattern where:
        # base+0: ə, base+1: u, base+2: i, base+3: a, base+4: e, base+5: ɨ, base+6: o
        variants = []
        for i in range(8):  # Include the 8th form if it exists (base+7)
            try:
                variant_code = base_code + i
                if 0xD800 <= variant_code <= 0xDFFF or variant_code > 0x10FFFF:
                    # Skip invalid Unicode code points
                    continue
                    
                variant = chr(variant_code)
                variants.append(variant)
            except:
                # If this variant doesn't exist, skip it
                pass
        
        return variants if variants else [base_char]
    except:
        return [base_char]

In [ ]:
def create_keyboard_ui():
    """
    Create a Tigrigna keyboard UI that matches the reference image
    
    Returns:
        tuple: (keyboard_widget, text_area)
    """
    # Create the text area for input
    text_area = widgets.Textarea(
        placeholder='ኣብዚ ይጻሓፉ...',  # Type here in Tigrigna
        layout=widgets.Layout(width='100%', height='100px', 
                             border='1px solid #ddd', 
                             padding='10px')
    )
    
    # Create the header area for input display and editing
    header = widgets.HTML(
        value='<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 30px;"></div>',
        layout=widgets.Layout(width='100%')
    )
    
    # Create a container for the vowel variant buttons (top row green buttons)
    variants_container = widgets.HBox([], 
                                     layout=widgets.Layout(
                                         width='100%', 
                                         background_color='#000000',
                                         padding='5px'))
    
    # Keep track of the currently selected base character
    current_base = [None]
    
    # Create the main keyboard container with black background
    keyboard_container = widgets.VBox([], 
                                     layout=widgets.Layout(
                                         width='100%',
                                         background_color='#000000',
                                         padding='5px'))
    
    # Create a label for the keyboard (showing "Tigrigna")
    keyboard_label = widgets.HTML(
        value='<div style="text-align: center; color: white; padding: 5px;">Tigrigna</div>',
        layout=widgets.Layout(width='100%')
    )
    
    # Define the Tigrigna keyboard layout based on the provided image
    # First row (numbers)
    row1 = ['፩', '፪', '፫', '፬', '፭', '፮', '፯', '፰', '፱', '፲']
    
    # Second row (first row of Tigrigna characters as seen in the image)
    row2 = ['ሀ', 'ሐ', 'ሠ', 'ረ', 'ሰ', 'ሸ', 'ቀ', 'ቐ', 'በ', 'ቨ']
    
    # Third row (second row of Tigrigna characters)
    row3 = ['ተ', 'ቸ', 'ኀ', 'ነ', 'ኘ', 'አ', 'ከ', 'ኸ', 'ወ', 'ዐ']
    
    # Fourth row (third row of Tigrigna characters) 
    row4 = ['ዘ', 'ژ', 'የ', 'ደ', 'ጀ', 'ገ', 'ጠ', 'ጨ', 'ጰ', 'ጸ']
    
    # Fifth row (fourth row of Tigrigna characters)
    row5 = ['ፀ', 'ፈ', 'ፐ', 'ቦ', 'ቱ', 'ሙ', 'ሉ', 'ኢ', 'ኣ', 'ኡ']
    
    # Create row structure - for the main keyboard
    keyboard_layout = [row1, row2, row3, row4, row5]
    
    # Function to create a character button with the right styling
    def create_char_button(char, width='34px'):
        btn = widgets.Button(
            description=char,
            layout=widgets.Layout(width=width, height='34px', margin='2px'),
            style=widgets.ButtonStyle(font_weight='bold', font_size='14px')
        )
        return btn
    
    # Function to update input text
    def update_input_text(text):
        text_area.value = text
        # Update the header text
        header.value = f'<div style="background-color: #f9f9f9; padding: 10px; border: 1px solid #ddd; margin-bottom: 10px; min-height: 30px;">{text}</div>'
    
    # Function to show vowel variants when a base character is clicked
    def show_variants(base_char):
        current_base[0] = base_char
        variants = generate_variants(base_char)
        
        # Create buttons for each variant with green background (like in the image)
        variant_buttons = []
        for i, variant in enumerate(variants[:3]):  # Show first 3 variants in top row
            btn = widgets.Button(
                description=variant,
                layout=widgets.Layout(width='33%', height='34px', margin='2px'),
                style=widgets.ButtonStyle(
                    button_color='#4CAF50',  # Green color from the image
                    font_weight='bold',
                    font_size='14px'
                )
            )
            
            # Add click handler for the variant button
            def on_variant_click(b, variant=variant):
                # Insert the selected variant at the cursor position
                cursor_pos = text_area.cursor_pos
                text = text_area.value
                
                # Add character at cursor position
                new_text = text[:cursor_pos] + variant + text[cursor_pos:]
                update_input_text(new_text)
                
                # Move cursor after the inserted character
                text_area.cursor_pos = cursor_pos + len(variant)
            
            btn.on_click(on_variant_click)
            variant_buttons.append(btn)
        
        # Update the variants container with the new buttons
        variants_container.children = tuple(variant_buttons)
    
    # Create rows for the keyboard with the proper layout
    keyboard_rows = []
    
    # Add the main rows from the keyboard layout
    for row in keyboard_layout:
        row_buttons = [create_char_button(char) for char in row]
        row_box = widgets.HBox(row_buttons, 
                              layout=widgets.Layout(
                                  width='100%', 
                                  justify_content='space-around',
                                  padding='2px'))
        keyboard_rows.append(row_box)
    
    # Create bottom row with special keys
    bottom_row_buttons = []
    
    # Backspace button
    backspace_btn = create_char_button('⌫')
    
    # Space button (wider)
    space_btn = widgets.Button(
        description=' ',
        layout=widgets.Layout(width='50%', height='34px', margin='2px'),
        style=widgets.ButtonStyle(font_weight='bold')
    )
    
    # Punctuation buttons
    colon_btn = create_char_button('፡')
    period_btn = create_char_button('።')
    
    # Enter button
    enter_btn = create_char_button('↵')
    
    # Add buttons to bottom row
    bottom_row_buttons = [backspace_btn, space_btn, colon_btn, period_btn, enter_btn]
    
    # Create the bottom row container
    bottom_row_box = widgets.HBox(bottom_row_buttons, 
                                 layout=widgets.Layout(
                                     width='100%', 
                                     justify_content='space-around',
                                     padding='2px'))
    
    keyboard_rows.append(bottom_row_box)
    
    # Add click handlers for character buttons
    for row_box in keyboard_rows[:-1]:  # Skip the bottom special row
        for button in row_box.children:
            def on_char_click(b):
                base_char = b.description
                show_variants(base_char)
                
                # Also insert the base character
                cursor_pos = text_area.cursor_pos
                text = text_area.value
                new_text = text[:cursor_pos] + base_char + text[cursor_pos:]
                update_input_text(new_text)
                text_area.cursor_pos = cursor_pos + len(base_char)
            
            button.on_click(on_char_click)
    
    # Add handlers for special buttons
    def on_backspace_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        
        if cursor_pos > 0:
            # Delete the character before the cursor
            new_text = text[:cursor_pos-1] + text[cursor_pos:]
            update_input_text(new_text)
            text_area.cursor_pos = cursor_pos - 1
    
    def on_space_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + ' ' + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + 1
    
    def on_punctuation_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + b.description + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + len(b.description)
    
    def on_enter_click(b):
        cursor_pos = text_area.cursor_pos
        text = text_area.value
        new_text = text[:cursor_pos] + '\n' + text[cursor_pos:]
        update_input_text(new_text)
        text_area.cursor_pos = cursor_pos + 1
    
    # Assign handlers to special buttons
    backspace_btn.on_click(on_backspace_click)
    space_btn.on_click(on_space_click)
    colon_btn.on_click(on_punctuation_click)
    period_btn.on_click(on_punctuation_click)
    enter_btn.on_click(on_enter_click)
    
    # Assemble the keyboard
    # Start with variants container (for showing the green variant buttons)
    keyboard_container.children = [variants_container]
    
    # Add the main keyboard rows
    keyboard_container.children = list(keyboard_container.children) + keyboard_rows
    
    # Add the keyboard label at the bottom (Tigrigna text)
    keyboard_container.children = list(keyboard_container.children) + [keyboard_label]
    
    # Create the complete UI with text area and keyboard
    complete_ui = widgets.VBox([
        header,
        text_area,
        keyboard_container
    ])
    
    return complete_ui, text_area

## 4. Spell Checking Functionality

Now let's implement the spell checking functionality.

In [ ]:
def levenshtein_distance(s1, s2):
    """Calculate the Levenshtein distance between two strings"""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def get_suggestions(word, dictionary, max_distance=2, max_suggestions=5):
    """Generate suggestions for a misspelled word"""
    suggestions = []
    
    for dict_word in dictionary:
        distance = levenshtein_distance(word, dict_word)
        if distance <= max_distance:
            suggestions.append((dict_word, distance))
    
    suggestions.sort(key=lambda x: x[1])
    return [word for word, _ in suggestions[:max_suggestions]]

def check_spelling(text, dictionary):
    """Check spelling of all words in a text and provide suggestions"""
    # Tokenize the text
    words = re.findall(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]+', text)
    words = [word.strip() for word in words if word.strip()]
    
    if not words:
        return []
    
    results = []
    for word in words:
        is_correct = word in dictionary
        results.append({
            'word': word,
            'is_correct': is_correct,
            'suggestions': [] if is_correct else get_suggestions(word, dictionary)
        })
    
    return results

def format_results(results):
    """Format spell check results as HTML"""
    html = '<div style="font-family: Arial, sans-serif; line-height: 1.6;">'
    html += '<h3>Spell Check Results:</h3>'
    
    if not results:
        html += '<p>No words to check.</p>'
    else:
        html += '<ul style="list-style-type: none; padding-left: 0;">'
        
        for item in results:
            word = item['word']
            is_correct = item['is_correct']
            suggestions = item['suggestions']
            
            if is_correct:
                html += f'<li style="margin-bottom: 10px;"><span style="color: green;">{word}</span> - ትኽክል</li>'
            else:
                html += f'<li style="margin-bottom: 10px;"><span style="color: red; text-decoration: underline;">{word}</span> - ጌጋ'
                
                if suggestions:
                    html += '<ul style="margin-top: 5px; margin-bottom: 5px;">'
                    html += '<li>ምትካእ፡</li>'
                    for suggestion in suggestions:
                        html += f'<li style="margin-left: 20px;">{suggestion}</li>'
                    html += '</ul>'
                else:
                    html += '<br>ምትካእ የለን።'
                    
                html += '</li>'
        
        html += '</ul>'
    
    html += '</div>'
    return html

## 5. Create and Display the Integrated UI

Now, let's create the integrated UI with the keyboard and spell checker.

In [ ]:
def create_integrated_ui(dictionary):
    """Create an integrated UI with keyboard and spell checker"""
    # Create the keyboard
    keyboard_ui, text_input = create_keyboard_ui()
    
    # Create an output area for spell check results
    output_area = widgets.Output()
    
    # Create a spell check button
    spell_check_button = widgets.Button(
        description='Check Spelling',
        button_style='success',
        icon='check',
        layout=widgets.Layout(width='200px', height='40px', margin='10px')
    )
    
    # Event handler for spell check button
    def on_spell_check(b):
        text = text_input.value
        
        with output_area:
            output_area.clear_output()
            
            if not text.strip():
                display(HTML('<p style="color: orange;">ብኽብረትኩም ዝኾነ ጽሑፍ ኣእትዉ።</p>'))
                return
            
            results = check_spelling(text, dictionary)
            display(HTML(format_results(results)))
    
    spell_check_button.on_click(on_spell_check)
    
    # Assemble the complete UI
    complete_ui = widgets.VBox([
        widgets.HTML('<h1>Tigrigna Spelling Checker</h1>'),
        widgets.HTML('<p>ብትግርኛ ጽሕፉ ወይ ኣብ ታሕቲ ዘሎ ፊደላት ተጠቐሙ፣ ድሕሪኡ "Check Spelling" ዝብል መልጎም ጠውቑ።</p>'),
        keyboard_ui,
        spell_check_button,
        output_area
    ])
    
    return complete_ui

# Create and display the integrated UI
if dict_exists:
    integrated_ui = create_integrated_ui(dictionary)
    display(integrated_ui)
else:
    print("Please make sure the dictionary file is available before proceeding.")

## Additional Features

### Adding New Words to the Dictionary

In [ ]:
def add_word_to_dictionary(word, dictionary_file='tigrigna_dictionary.txt'):
    """Add a new word to the Tigrigna dictionary"""
    if not word or not word.strip():
        print("Please enter a valid word to add.")
        return False
    
    # Check if the word contains Tigrigna characters
    if not re.search(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]', word):
        print("The word must contain Tigrigna characters.")
        return False
    
    try:
        # Load existing dictionary to check if word already exists
        with open(dictionary_file, 'r', encoding='utf-8') as file:
            existing_words = {line.strip() for line in file if line.strip()}
        
        if word in existing_words:
            print(f"The word '{word}' is already in the dictionary.")
            return False
        
        # Add the word to the dictionary file
        with open(dictionary_file, 'a', encoding='utf-8') as file:
            file.write(f"\n{word}")
        
        print(f"Successfully added '{word}' to the dictionary.")
        return True
    except Exception as e:
        print(f"Error adding word to dictionary: {e}")
        return False

# Create UI for adding words to the dictionary
def create_dictionary_update_ui():
    """Create UI for adding words to the dictionary"""
    word_input = widgets.Text(
        placeholder='Enter a Tigrigna word',
        description='New Word:',
        layout=widgets.Layout(width='300px')
    )
    
    add_button = widgets.Button(
        description='Add to Dictionary',
        button_style='primary',
        icon='plus'
    )
    
    output = widgets.Output()
    
    def on_add_button_click(b):
        word = word_input.value.strip()
        
        with output:
            output.clear_output()
            if add_word_to_dictionary(word):
                # Clear the input field after successful addition
                word_input.value = ''
    
    add_button.on_click(on_add_button_click)
    
    ui = widgets.VBox([
        widgets.HTML('<h3>Add New Words to Dictionary</h3>'),
        widgets.HBox([word_input, add_button]),
        output
    ])
    
    return ui

# Display the dictionary update UI
if dict_exists:
    dictionary_update_ui = create_dictionary_update_ui()
    display(dictionary_update_ui)